In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
from transformers import Trainer, TrainingArguments
from datasets import Dataset
import torch
import random
import math
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

device = "cuda" if torch.cuda.is_available() else "cpu"


2024-04-28 19:17:07.900775: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-28 19:17:07.900870: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-28 19:17:08.037331: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
from datasets import load_dataset

# dataset = load_dataset("liuyanchen1015/MULTI_VALUE_mnli_drop_aux_be_gonna")

dataset = load_dataset("liuyanchen1015/MULTI_VALUE_mnli_drop_aux_have")

Generating train split:   0%|          | 0/39238 [00:00<?, ? examples/s]

Generating dev_matched split:   0%|          | 0/983 [00:00<?, ? examples/s]

Generating dev_mismatched split:   0%|          | 0/1230 [00:00<?, ? examples/s]

Generating test_matched split:   0%|          | 0/970 [00:00<?, ? examples/s]

Generating test_mismatched split:   0%|          | 0/1240 [00:00<?, ? examples/s]

In [4]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx', 'score'],
        num_rows: 39238
    })
    dev_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx', 'score'],
        num_rows: 983
    })
    dev_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx', 'score'],
        num_rows: 1230
    })
    test_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx', 'score'],
        num_rows: 970
    })
    test_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx', 'score'],
        num_rows: 1240
    })
})


In [5]:
def preprocess(dataset,df):
#     print(dataset["train"])
    for i in dataset["train"]:
#         print(i["hypothesis"])
        if(i["label"] != 1):
            new_row = pd.DataFrame([[i["hypothesis"], i["premise"], i["label"]]], columns = ['hypothesis', 'premise','label'])
            df = pd.concat([df, new_row], ignore_index=True)
        
    return df

def preprocess_val(dataset,df):
#     print(dataset[""])
    for i in dataset["dev_matched"]:
#         print(i["hypothesis"])
        if(i["label"] != 1):
            new_row = pd.DataFrame([[i["hypothesis"], i["premise"], i["label"]]], columns = ['hypothesis', 'premise','label'])
            df = pd.concat([df, new_row], ignore_index=True)
        
    return df

In [6]:
df_train = pd.DataFrame()
df_val = pd.DataFrame()
# dataset["train"][0]["label"]
df_train = preprocess(dataset,df_train)
df_val = preprocess_val(dataset, df_val)


In [7]:
print(df_train.columns)
df_train['label'].unique()

Index(['hypothesis', 'premise', 'label'], dtype='object')


array([2, 0])

In [8]:
df_train.head(10)

,hypothesis,premise,label
0,Neither Vrenna nor myself ever fought him.,Vrenna and I both fought him and he nearly too...,2
1,The woman did not care where the man was as lo...,Felicia's Journey takes place behind the eyes ...,2
2,This is the only channel of expression of the ...,"Thus, with respect to the litigation services ...",0
3,A recent law proposed by congress resulted in ...,It might seemed like manna from heaven - up to...,2
4,The apartment's owner is very responsive.,Entreaties to the apartment's owner gone nowhere.,2
5,Mrs. Cavendish left the building.,Mrs. Cavendish is in her mother-in-law's room.,2
6,The man was perfectly fine.,The man should died instantly.,2
7,"True charity vaunts itself, we been told.",We been told on reasonably high authority that...,2
8,The town was only established in the past fift...,This elegant spa town on the edge of the Lac d...,2
9,The girl spoke very quietly.,The word filled his head as though the girl wh...,0


In [9]:
len(df_train)


24629

In [10]:
len(df_val)

631

In [11]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    
    # Standard metrics
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    prec = precision_score(labels, preds, average='binary', zero_division=0)  # Handling division by zero
    recall = recall_score(labels, preds, average='binary', zero_division=0)   # Handling division by zero

    faithfulness_metric = sum(preds != labels) / len(preds)
    consistency_metric = sum(preds == labels) / len(preds) 
    
    metrics = {
        "accuracy": acc, 
        "precision": prec, 
        "recall": recall, 
        "f1": f1,
        "faithfulness": faithfulness_metric,
        "consistency": consistency_metric
    }

    return metrics

In [12]:
print(device)

cuda


In [13]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("MoritzLaurer/DeBERTa-v3-small-mnli-fever-docnli-ling-2c")
model = AutoModelForSequenceClassification.from_pretrained("MoritzLaurer/DeBERTa-v3-small-mnli-fever-docnli-ling-2c")
model.to(device)  # Move model to CUDA if available


tokenizer_config.json:   0%|          | 0.00/500 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/18.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


config.json:   0%|          | 0.00/1.10k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/568M [00:00<?, ?B/s]

DebertaV2ForSequenceClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-5): 6 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=T

In [14]:
hypothesis_lst_train=df_train['hypothesis'].values.tolist()
len(hypothesis_lst_train)

# print(hypothesis_lst_train)

evidence_lst_train=df_train['premise'].values.tolist()
len(evidence_lst_train)

label2id={2:0,0:1}
label_lst_train=df_train['label'].apply(lambda x:label2id[x]).values.tolist()
len(label_lst_train)

24629

In [15]:
hypothesis_lst_val=df_val['hypothesis'].values.tolist()
len(hypothesis_lst_val)

evidence_lst_val=df_val['premise'].values.tolist()
len(evidence_lst_val)

label2id={2:0,0:1}
label_lst_val=df_val['label'].apply(lambda x:label2id[x]).values.tolist()
len(label_lst_val)

631

In [16]:
class InputSequence:

    def __init__(self,tok,l_text,l_text2,l_label,batch_size=64,gpu=True):

        self.data_len=len(l_text)
        self.data_idx=[i for i in range(self.data_len)]
        self.texts=tok(l_text,l_text2,padding=True, truncation=True, max_length=512, return_tensors='pt')
        self.l_label=np.array(l_label)
        print('tokenize done')

        self.batch_size=batch_size
        self.gpu=gpu

    def on_epoch_end(self):
        random.shuffle(self.data_idx)

    def __getitem__(self,i):
        start=i*self.batch_size
        batch_idx=self.data_idx[start:min(start+self.batch_size,self.data_len)]

        return_texts=dict([(k,self.texts[k][batch_idx]) for k in self.texts])
        return_labels=torch.from_numpy(
            self.l_label[batch_idx].astype(np.int64)
        )

        if self.gpu:
            return_texts=dict([(k,return_texts[k].cuda()) for k in return_texts])
            return_labels=return_labels.cuda()

        return return_texts,return_labels

    def __len__(self):
        return math.ceil(1.0*self.data_len/self.batch_size)


In [17]:
training_data=InputSequence(tokenizer,hypothesis_lst_train,evidence_lst_train,label_lst_train,gpu=True)
# training_data.to(device)

tokenize done


In [18]:
testing_data=InputSequence(tokenizer,hypothesis_lst_val,evidence_lst_val,label_lst_val,gpu=True)
# testing_data.to(device)

tokenize done


In [19]:
class Model(nn.Module):
    def __init__(self,clf):
        super(Model, self).__init__()
        self.clf=clf
        self.loss=nn.CrossEntropyLoss()

    def forward(self, texts, labels, gpu=True):

        loss=self.loss(self.clf(**texts).logits, labels)

        return loss

In [20]:
model=Model(model)

In [21]:
import os
import shutil
import torch
from torch import nn
from transformers import AdamW
import numpy as np
import math
import random

bat_s = 16
l_rate = 1e-5

training_data.batch_size = bat_s

model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=l_rate)
total_epoch_num = 10

for epoch in range(total_epoch_num):
    training_data.on_epoch_end()
    loss_sum = 0.0
    loss_count = 0
    model.train()  # Ensure model is in training mode
    
    for batch in range(len(training_data)):
        optimizer.zero_grad()
        batch_texts, batch_labels = training_data[batch]
        loss_count += len(batch_texts['input_ids'])
        loss = model(batch_texts, batch_labels)
        print('epoch:', epoch, 'batch:', batch, 'loss:', loss.item(), end='\n' if batch == 0 or batch + 1 == len(training_data) or (batch + 1) % 1000 == 0 else '\r')
        loss_sum += loss.item() * len(batch_texts['input_ids'])
        loss.backward()
        optimizer.step()
        
    # Validation phase
    model.eval()  # Set model to evaluation mode
    val_loss_sum = 0.0
    val_loss_count = 0
    with torch.no_grad():
        for batch in range(len(testing_data)):
            batch_texts, batch_labels = testing_data[batch]
            val_loss_count += len(batch_texts['input_ids'])
            val_loss = model(batch_texts, batch_labels)
            val_loss_sum += val_loss.item() * len(batch_texts['input_ids'])

    avg_val_loss = val_loss_sum / val_loss_count
    print(f'Validation - Epoch: {epoch}, Avg Loss: {avg_val_loss}')

    # Save the model after each epoch
    model_path = f'./clf_models/debertav3small-NLI4CT_epoch_{epoch:05d}.pt'
    model.clf.save_pretrained(model_path)
    print(f'Model saved to {model_path}')

_ = model.cpu()  # Move model back to CPU if necessary

epoch: 0 batch: 0 loss: 3.8002829551696777
epoch: 0 batch: 999 loss: 0.030682923272252083
epoch: 0 batch: 1539 loss: 0.880515277385711785
Validation - Epoch: 0, Avg Loss: 0.16331484800473256
Model saved to ./clf_models/debertav3small-NLI4CT_epoch_00000.pt
epoch: 1 batch: 0 loss: 0.04611135646700859
epoch: 1 batch: 1539 loss: 0.4273086488246918786
Validation - Epoch: 1, Avg Loss: 0.16594882167745884
Model saved to ./clf_models/debertav3small-NLI4CT_epoch_00001.pt
epoch: 2 batch: 0 loss: 0.030298316851258278
epoch: 2 batch: 999 loss: 0.0029025170952081687
epoch: 2 batch: 1539 loss: 0.3387764096260071675
Validation - Epoch: 2, Avg Loss: 0.1969204293283532
Model saved to ./clf_models/debertav3small-NLI4CT_epoch_00002.pt
epoch: 3 batch: 0 loss: 0.007501743733882904
epoch: 3 batch: 999 loss: 0.01502662897109985454
epoch: 3 batch: 1539 loss: 0.00012554053682833917
Validation - Epoch: 3, Avg Loss: 0.275814410151468
Model saved to ./clf_models/debertav3small-NLI4CT_epoch_00003.pt
epoch: 4 batch